### Imports

In [ ]:
import datetime
import os

import geopandas as gpd
import pandas as pd
import partridge as ptg

### Read in the GTFS Files
Specify the path to the GTFS folder.

In this case, the script is selecting the busiest day of the GTFS to use as it's guide.

In [ ]:
gtfs_feed = 'carta_2019_08_15'
gtfs_path = os.path.join('..', '..', 'data', 'gtfs', gtfs_feed)

service_date = datetime.date(2019, 10, 16)
service_ids = ptg.read_service_ids_by_date(gtfs_path)[service_date]

### The busiest day for the two 2019 CARTA feeds are holidays (Memorial Day 2019 and MLK 2020)
#service_date, service_ids = ptg.read_busiest_date(gtfs_path)

print('Selected Date: {}'.format(service_date))

view = {
    'trips.txt': {'service_id': service_ids},
    'routes.txt': {'route_id': '28'}
}

feed = ptg.load_geo_feed(gtfs_path, view)

### Tag each trip from the GTFS with a departure time from it's first stop
This will be used to determine which model time period it is related.

In [ ]:
first_departure = feed.stop_times.iloc[feed.stop_times.groupby('trip_id')['departure_time'].idxmin()]
trips = pd.merge(feed.trips, first_departure[['trip_id', 'departure_time']], on='trip_id', how='left')

### Merge and Export
- Merge (Spatial Attribute Join) GTFS Shapes and Trips  
- Specify the DBF Schema and Export to Shapefile 

In [ ]:
for tod in ['AM', 'OP', 'MD']:

    # DEFAULT TO AM
    departure_window = [6 * 3600, 9 * 3600 - 1]

    ## Departure windows are in seconds from midnight (Partridge does this when times are loaded.)
    if tod == 'OP':
        departure_window = [9 * 3600, 15 * 3600 - 1]
    if tod == 'PM':
        departure_window = [15 * 3600, 18 * 3600 - 1]

    trips_tod = trips[trips['departure_time'].between(*departure_window)].copy()

    trip_fields = ['route_id', 'trip_headsign', 'direction_id', 'shape_id']
    route_shapes = feed.shapes.merge(trips_tod.drop_duplicates(trip_fields)[trip_fields],
                                     on='shape_id', how='inner')


    schema = {'geometry': 'LineString',
              'properties':
                  {'shape_id': 'str:15', 
                   'route_id': 'str:25',
                   'trip_headsign': 'str:25',
                   'direction_id': 'int'
                  }
             }

    route_shapes.to_file(os.path.join('..', '..', 'data', 'gtfs', 'route_shapes_{}_{}.shp'.format(gtfs_feed, tod)),
                         schema=schema)